# Databases 2

### OR

## Hash tables and their use in Databases

### Why do you care?

- you will see many databases in your career
- you will code SQL
- you will have to deal with systematic storage of structured and unstructured data at some point
- and you will have to access this data from it storage

### Some databases

- **PostgreSQL** and **sqlite** are relational database management: tuples in data rows and strictly enforced column types. Schema.
- **Riak** distributed, fault-tolerant key-value database, inspired by Amazon Dynamo, low latency. A storage engine for it is *bitcask*, based on the deas from the database we wrote last time. No schema.
- **HBase**: large database (big data), columnar, scalable, corenerstone for OLAP, fault tolerant, used with Hadoop, based on Google's BigTable
- **Mongodb** is a document database, storing JSON documents, schemaless
- **Neo4j** is a graph database, stores relationship or edges between nodes. Great for semantic web type stuff
- **redis** : data structure server, fast, transactional, can be used for queue, pub/sub, sorted sets, bloom filters, expiration, replication, but not surability

### Polyglot persistence

An example from 7 databases in 7 weeks:

>We want to store a list of musical band names, the artists who per- formed in those bands, and any number of roles each artist played in the band, from lead singer to backup keytar player. Each of three databases— Redis, CouchDB, and Neo4j—will handle a different aspect of our band management system.

>Redis plays three important roles in our system: to assist in data populating CouchDB, as a cache for recent Neo4j changes, and as a quick lookup for partial value searches. Its speed and ability to store multiple data formats make it well suited for population, and its built-in expiry policies are perfect for handling cached data.

>CouchDB is our system of record (SOR), or authoritative data source. CouchDB’s document structure is an easy way to store band data with nested artist and role information, and we will take advantage of the Changes API in CouchDB to keep our third data source in sync.

>Neo4j is our relationship store. Although querying the CouchDB SOR directly is perfectly reasonable, a graph datastore allows us a simplicity and speed in walking node relationships that other databases have a difficult time matching. We’ll store relationships between bands, band members, and the roles the members play.

### From last time

![](https://dl.dropboxusercontent.com/u/75194/hashmaplog.png)

- in the dict in memory, store a file offset instead
- this "HEAP" file is an append only file, and thus also acts as a "LOG"(WAL)
- if you update, simply append a new entry and change the offset in the dict

- this is what bitcask in Riak does ( http://basho.com/wp-content/uploads/2015/05/bitcask-intro.pdf)
- what if file becomes too large?

On disk:

![](https://dl.dropboxusercontent.com/u/75194/riak1.png)

In memory:
![](https://dl.dropboxusercontent.com/u/75194/riak2.png)

(from riak bitcask intro at http://basho.com/wp-content/uploads/2015/05/bitcask-intro.pdf )

- deletion is done by writing a tombstone record
- break the file into segments. Each segment, once written, the kv pairs are never changed. 

How do we not run out of disk space>

- run compaction to throw away dupes from segments and merge them into a NEW file; delete old files
- tombstone means merging process can discard previous values

![](https://dl.dropboxusercontent.com/u/75194/compactionmerge.png)


- Process model: only one writer thread. Many readers.
- Maintain a hashmap per segment and search these in order from most recent to least recent.
- bitcask will store hashmap snapshots

What are these hashmaps we talk about?


### Hash table basics

Hash tables are a great for making dictionaries.

The basic idea is that once u have an index, looking something up in an array is O(1). 
Hash tables consist of four parts:
1. Keys
2. Values
3. A hash function
4. A storage array

![Hash Table Diagram](hash_overview.png)

The basic idea is that a hash function provides a constant-time approximation of where to find or store a particular key-value pair. In the common case, the key-value pair can be found or stored at that location. In the uncommon case, the function returned the same value for some other key, so some other location must be searched or used to complete the operation. When a hash function returns the same value for two distinct keys, it is called a collision, and the mechanism to recover is called collision resolution.

Indeed, there does not even need to be a set of values. It might just be a bunch of keys. In such a way, a hash table can be used to implement a set: the same values hashes to the same index so there are no repeats.

What is this hash function? It is a function that, given your key, and the multitude of possible keys, maps the key to an integer representing an index. For example, suppose your keys are integers k:

$$f(k) = k\,mod\,11$$

![](http://interactivepython.org/courselib/static/pythonds/_images/hashtable2.png)

(example from interactivepython.org)

A perfect hash function would map each key to a separate index. Hard to do and certainly not possible when u dont know the keys ahead of time. In the hash function above, 66 and 44 both map to 0.

Suppose we were puuting in (54,26,93,17,77,31,44,55,20). Then we'd have **collision** when we'd try to put 44 in.

One way to deal with this is to use open addressing:

![](http://interactivepython.org/courselib/static/pythonds/_images/linearprobing1.png)

Here we put the 44 into the next available spot, say after the 77. This method does lead to clustering but is often used. Then 20 should go in slot 9 but thats used by 31 so we resort to *linear probing* from the beginning to find an empty slot and put it there.

We can skip finite numbers while probing, or even have a probing function. The point is that the process must be repeatable so we can "get" the value for a key later.

The other way to do this is to create a linked list at each array slot and add all collided keys into that linked list. This is called **chaining**.

![](http://interactivepython.org/courselib/static/pythonds/_images/chaining.png)

In general you will want a hash function to map a key to a large integer. If $\alpha$is the size of the alphabet using which a string S is written, and char(c) is a function mapping each letter in the alphabet to a unique integer from 0 to $\alpha  - 1$.

Then,

$$H(S) = \sum_{i=0}^{sz(S) - 1} \alpha^{sz(S) - (i+1)} char(i)$$

maps each string to a unique but large integer by treating each character as a digit in a base-$\alpha$ numerical system.

These intergers will overflow our size (say) m array of slots in the hash table, so we now do:

$$H(S)\,mod\,m$$. One can show that if m is chosen well (for example a large prime not too close to $2^{\alpha} - 1$, the resulting hash values will be fairly uniformly distributed.
 
To avoid collisions, chaining seems to be the natural thing to do, but devotes considerable memory to pointers, memory better used at making m larger. But open addressing means that we must systematically do some multi hop searching on a run of indexes.  

So implicitly we have asked for:

1. If two objects are equal, then their hashes should be equal.
2. If two objects have the same hash, then they are likely to be the same object. (modulo collisions)

With all this you might think that the dunder `__hash__` delegated to by python for the `hash` function is a hash function that tries to uniformy distribute its contents. This turns out to be not be true, a decision made from the practical observation that often hash table keys have some "systematic" ordering

In [26]:
hash(1), hash(2), hash(3)

(1, 2, 3)

In [27]:
hash('a'), hash('b'), hash('c')

(-5979502230166123623, -7224185177662446553, -7128687885686014242)

In [28]:
hash([])

TypeError: unhashable type: 'list'

The python hashing implementation can be seen at [dictobject.c](http://svn.python.org/projects/python/trunk/Objects/dictobject.c). There are some great notes on the hashing algorithm there, from which I quote:

```c
Major subtleties ahead:  Most hash schemes depend on having a "good" hash
function, in the sense of simulating randomness.  Python doesn't:  its most
important hash functions (for strings and ints) are very regular in common
cases:

>>> map(hash, (0, 1, 2, 3))
[0, 1, 2, 3]
>>> map(hash, ("namea", "nameb", "namec", "named"))
[-1658398457, -1658398460, -1658398459, -1658398462]
>>>

This isn't necessarily bad!  To the contrary, in a table of size 2**i, taking
the low-order i bits as the initial table index is extremely fast, and there
are no collisions at all for dicts indexed by a contiguous range of ints.
The same is approximately true when keys are "consecutive" strings.  So this
gives better-than-random behavior in common cases, and that's very desirable.

OTOH, when collisions occur, the tendency to fill contiguous slices of the
hash table makes a good collision resolution strategy crucial.  Taking only
the last i bits of the hash code is also vulnerable:  for example, consider
[i << 16 for i in range(20000)] as a set of keys.  Since ints are their own
hash codes, and this fits in a dict of size 2**15, the last 15 bits of every
hash code are all 0:  they *all* map to the same table index.

But catering to unusual cases should not slow the usual ones, so we just take
the last i bits anyway.  It's up to collision resolution to do the rest.  If
we *usually* find the key we're looking for on the first try (and, it turns
out, we usually do -- the table load factor is kept under 2/3, so the odds
are solidly in our favor), then it makes best sense to keep the initial index
computation dirt cheap.

The first half of collision resolution is to visit table indices via this
recurrence...
```

A good resource to see the whole story is http://www.laurentluce.com/posts/python-dictionary-implementation/ . Knowing C will help!

### Python dictionaries

Python `dict`s are hash tables. We're going to understand how they operate internally.

As we've already discussed, Python data structures are largely defined by the interfaces they present. There are two interfaces we care about here:

 - Objects that act like dictionaries
 - Objects that can act as key for dictionaries
 
Just like sequences and iterables, the interface for Python dictionaries can be described by a small set of special methods. The easiest way to understand this is to match Python statements involving dictionaries with the special methods they call.

In [2]:
dictionary = {}
### All of these pairs of statements are equivalent.

# Setting an item
dictionary['key1'] = 'value1'
dictionary.__setitem__('key2', 'value2')

# Getting an item
x = dictionary['key1']
x = dictionary.__getitem__('key2')

# Get the size of an item
len(dictionary)
dictionary.__len__()

# Iterating over a dictionary
[key for key in dictionary]
[key for key in dictionary.__iter__()]

# Deleting an item
del dictionary['key1']
dictionary.__delitem__('key2')

# Checking at item (OPTIONAL)
'key1' in dictionary
dictionary.__contains__('key2')
for key in dictionary: # Fallback behavior, if __contains__ is not defiend
    if key=='key1':
        True
else: # You can have else blocks on for-loops in Python
    False

# Default values (OPTIONAL)
# Normal dicts do not implement this feature
# x = dictionary['missing-key']
# x = dictionary.__missing__('missing-key')

### Dictionary keys

Dictionary keys in Python can be any object that satisfies the interface convention.

A common misconception is that dictionary keys must be immutable.

In actuality, Python dicts define an interface just like everything else. There are two relavent special methods:

1. `__hash__`: returns an integer. This will eventually get converted by Python to an index.
2. `__eq__`: compares two objects for equality. This is how collisions are detected.

In general, immutable built-in types (like numbers and tuples) have `__hash__` methods defined, and built-in mutable types (like lists and dicts) have `__hash__` set to `None`.

Let's see some examples.



 Two things to remember:

 - User-defined classes inherit a `__hash__` from `object` which returns a hash based on the identity of the object.
 - The implementation of `__eq__` inherited from `object` compares the identity of two objects.

In [29]:
class UtilityMethodsMixin(): 
    def __init__(self, v):
        self.set(v)
    def set(self, v):
        self.v = v
    def __repr__(self):
        return '<OBJ|id:'+str(id(self))+'|v:'+str(self.v)+'>'

First up, `hash(A)==hash(A)`. Hashes should be consistent.

In [30]:
import random
class InconsistentHashable(UtilityMethodsMixin):
    def __hash__(self):
        obtained = random.randint(0,1000)
        print("ob", obtained)
        return obtained

In [10]:
A = InconsistentHashable(v=0)
print(A)
print(hash(A), hash(A), hash(A))
print('----')
dictionary = {}
dictionary[A] = 'x'
dictionary[A] = 'y'
dictionary[A] = 'z'
print(dictionary)
print( A in dictionary )

<OBJ|id:4363907480|v:0>
ob 711
ob 679
ob 108
711 679 108
----
ob 389
ob 34
ob 544
{<OBJ|id:4363907480|v:0>: 'z', <OBJ|id:4363907480|v:0>: 'y', <OBJ|id:4363907480|v:0>: 'x'}
ob 30
False


Next: Normally, `hash(A)!=hash(B)` if `A!=B`. We normally want to *avoid* collisions in hash tables.

In [11]:
class NormalHashable(UtilityMethodsMixin):
    def __hash__(self):
        return hash(id(self))
    def __eq__(self, other):
        return self.v==other.v
class SlowHashable(UtilityMethodsMixin):
    def __hash__(self):
        return 0
    def __eq__(self, other):
        return self.v==other.v

In [12]:
%%timeit
dictionary = {}
for i in range(1000):
    A = NormalHashable(i)
    dictionary[A] = i

1000 loops, best of 3: 1.08 ms per loop


In [13]:
%%timeit
dictionary = {}
for i in range(1000):
    A = SlowHashable(i)
    dictionary[A] = i

1 loop, best of 3: 164 ms per loop


Constantly causing collisions *really* slows things down.

Finally, the strange one: "`A==B` implies `hash(A)==hash(B)`. If two objects are equal, they should always hash to the same value.

In [16]:
class StrangeHashable(UtilityMethodsMixin):
    def __hash__(self):
        return id(self)%10
    def __eq__(self, other):
        return self.v==other.v

In [17]:
A = [A for A in [StrangeHashable(v=0) for _ in range(0,50)] if hash(A)==0][0] # Find an instance which hashes to 0
B = [B for B in [StrangeHashable(v=0) for _ in range(0,50)] if hash(B)==0][0] # Find an instance which hashes to 0
C = [C for C in [StrangeHashable(v=0) for _ in range(0,50)] if hash(C)==2][0] # Find an instance which hashes to 2

print(A, B, C)
print('(A==B)',A==B, ' (hash(A)==hash(B))',hash(A)==hash(B))
print('(B==C)',B==C, ' (hash(B)==hash(C))',hash(B)==hash(C))

dictionary = {}
dictionary[A] = 'a'
print(dictionary)
dictionary[B] = 'b'
print(dictionary)
dictionary[C] = 'c'
print(dictionary)



<OBJ|id:4363910000|v:0> <OBJ|id:4363907760|v:0> <OBJ|id:4363907872|v:0>
(A==B) True  (hash(A)==hash(B)) True
(B==C) True  (hash(B)==hash(C)) False
{<OBJ|id:4363910000|v:0>: 'a'}
{<OBJ|id:4363910000|v:0>: 'b'}
{<OBJ|id:4363910000|v:0>: 'b', <OBJ|id:4363907872|v:0>: 'c'}


** TL;DR: **

 - `__hash__` and `__eq__` govern `dict` behavior in Python.
 - If you're going to implement a class which can act like a dictionary key:
  - `hash(A)==hash(A)`
  - Normally, `hash(A)!=hash(B)` if `A!=B`
  - `A==B` implies `hash(A)==hash(B)`
  
More such fun at http://www.asmeurer.com/blog/posts/what-happens-when-you-mess-with-hashing-in-python/, which also notes that:

>in Python 3, if you override `__eq__`, it automatically sets `__hash__` to None, making the object unhashable. You need to manually override `__hash__` to make it hashable again. But that's as far as Python goes in enforcing these rules, as we will see below. In particular, Python will never actually check that your `__hash__` actually agrees with your `__eq__`.

### LSM Tree

Why is appending in our Riak'ish database a good strategy?

- sequential writes are faster than random writes on both old disks and SSDs
- append-only or immutable files are safer in a crash when updating a value
- but hashmaps perform poorly on disk (random IO)
- and range queries are not efficient

This leads us to:


The big change is that we now write our heap files in SORTED key-value order.

What? Does this not break our nice LOG format? And the ability to use sequential files?

### LSM Tree

1. when a write comes in add it first to the WAL.
2. then to an in-memory balanced binary tree tree structure, called a memtable, for example a red-black tree. The WAL above's sole purpose is to reconstruct this tree in the face of a crash
3. At some duration or number-of-writes threshold we write the tree into a SSTable (sorted string table) file. This becomes the most recent segment of the database, and the memtable is cleared out for reuse. Because the memtable was sorted, the sstable is sorted as well. A bitcask style key-offset in-memory hashmap is used in a sparse way to remember some keys. The rest can be found by linear scanning from that offset onwards
4. we run merging and compaction in the background in the spirit of a mergesort and to discard overwritten or deleted values.


#### Compaction and merging

![](https://dl.dropboxusercontent.com/u/75194/lsmmerge.png)

#### In memory sparse hashmap

![](https://dl.dropboxusercontent.com/u/75194/lsmsparseindex.png)

#### How does it work?

```
    BF--->memtable--->in-memory sparse-hashmaps--->
        sstable segment--->sstable segment--->....
```

Searches go from memtable to these segments, one by one based on recent-ness, with a bloom filter being used to tell if the key was never there. 

### Bloom Filter

In the LSM tree, a bloom filter takes care of asking, is the key in our system at all. Its a strange data structure: it can answer in the affirmative that something is not in it, but only probabilistically that something is in it.

What is a bloom filter?

Basically its a hash table. The hash function computes a index, and the value corresponding to that index is a bit either set to 1 or 0. (In python this may be done by installing the bitarray or bitstring module).

One might ask, why not just use a set? The answer is, what if we want, as in the LSM tree, only one of the answers, and what if we want this to me a compact in-memory data structure, also true for a LSM tree. In a similar way, browsers use this to blacklist malaware sites.

```
pip install bitarray
pip install mmh3
```

There are two key numbers in a bloom filter

1. the number of hash functions: can range from a few to a dozen. These could be different functions or the same functions with different seeds. A good hash function spreads keys uniformly
2. thie size of the array: the number of bits we have. This is application dependent

The math for a bloom filter can be found on wikipedia (https://en.wikipedia.org/wiki/Bloom_filter) : we will not go into it here.

We'll reproduce an example from http://www.maxburstein.com/blog/creating-a-simple-bloom-filter/ where the murmur hash funcrion is used (coming up with hash functions yourself is not easy).


In [33]:
from bitarray import bitarray
import mmh3
 
class BloomFilter:
    
    def __init__(self, size, hash_count):
        self.size = size
        self.hash_count = hash_count
        self.bit_array = bitarray(size)
        self.bit_array.setall(0)
        
    def add(self, string):
        for seed in range(self.hash_count):
            result = mmh3.hash(string, seed) % self.size
            self.bit_array[result] = 1
        
    def lookup(self, string):
        for seed in range(self.hash_count):
            result = mmh3.hash(string, seed) % self.size
            if self.bit_array[result] == 0:
                return "Nope"
        return "Probably"

In [34]:
bf = BloomFilter(100, 4)
bf.add("rahul")
bf.add("rohan")
bf.add("anthony")
bf.add("nicholas")

In [35]:
bf.lookup("rahul")

'Probably'

In [36]:
bf.lookup("sophie")

'Nope'

Range queries are supported because we keep things in sort order.

- LevelDB, RocksDB (both embedded), Cassandra, HBase, Lucene are all examples of this append-to-log like approach. Cassandra, HBase and Leveldb use LSM Trees. Lucene's index  uses a key-value structure where the key is a wordand value is the list of docs containing the word. The index is kept in SSTable-like sorted files, 
and merged in the background as needed.
- The second school is the update-in-place school, using overwritable fixed size pages. Btrees are the major example here, but even they can be treated in a functional append-only style, as is done in lmdb and boltdb.


More about NOSQL dbs at https://en.wikipedia.org/wiki/NoSQL .

### B-tree vs LSM tree

- critical difference: btrees update in-place!
- comparable on random reads
- compaction can affect performance in LSM trees
- LSM tree good on random writes as it makes the writes sequential
- LSM true and b-tree are both used for transactions
- B-tree very good for transactions; at most one place where things are: you just need to lock a section of the tree. But concurrency can be complex as we shall see later and in-place updates are dangerous.

## Transaction Processing or Analytics?

- Also known as OLTP vs OLAP/Warehousing
- small query size vs aggregates over large ones
- random writes from user input vs ordered ETL/stream
- end user (amazon site) vs analyst (you)
- GB to TB vs TB to PB

![](https://dl.dropboxusercontent.com/u/75194/ETL.png)

(from designing data intensive applications)

### Row Oriented Storage

- heapfile or clustered index is a set of rows
- we'll see details of this storage soon
- index could be a tree with appropriate pointer to heapfile offset


![](https://www.simple-talk.com/iwritefor/articlefiles/1844-f4cc85b0-9ddb-44cc-93ef-a742fcc4f279.jpeg)

### Column oriented storage


- store values from each column together in separate storage
- lends itself to compression with bitmap indexes and run-length encoding
- this involves choosing an appropriate sort order
- the index then can be the data (great for IN and AND queries): there is no pointers to "elsewhere"
- compressed indexes can fit into cache and are usable by iterators
- bitwise AND/OR can be done with vector processing
- several different sort orders can be redundantly stored
- writing is harder: updating a row touches many column files
- but you can write an in-memory front sorted store (row or column), and eventually merge onto the disk (Binary Tree!) (Vertica does this)

![](https://www.simple-talk.com/iwritefor/articlefiles/1844-4e2482bb-aaff-4ebd-8900-1946560479af.jpeg)

(images from https://www.simple-talk.com/sql/database-administration/columnstore-indexes-in-sql-server-2012/)

Examples: Vertica, monetdb, hbase, cassandra, etc. SQL server has a columnar mode.

Some thoughts on usage:

- relational databases are good for structured but bad for hierarchicval and unstructured data
- key value stored have no additional index (the database is the index) and are very horizontally scalable which means i can add more nodes to my system to improve performance. bad for joins and scans.
- columnar databases are good for big data, horizontal scaling
- document databases are good for highly variable and unstructured data, that is nevertheless queried only one way like a k-v store
- graph databases good for highly connected data such as social networks and semantic web where there are lots of relationships.